Load the "MainProcess.xes" file

In [2]:
import os
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import pandas as pd

# Directory with your .xes files
xes_directory = os.path.join(os.getcwd(), "20130794", "Cleaned Event Log")

# Output directory for models or visuals
output_directory = os.path.join(os.getcwd(), "output")
os.makedirs(output_directory, exist_ok=True)

# Process MainProcess.xes file
filename = "MainProcess.xes"
file_path = os.path.join(xes_directory, filename)
print(f"Processing {filename}")

try:
    log = xes_importer.apply(file_path)
    print("\n\n")
    print(f"Imported {filename} with {len(log)} traces.")
    
    # Print important information about the log
    print(f"Number of events: {sum(len(trace) for trace in log)}")
    activities = set(event["concept:name"] for trace in log for event in trace if "concept:name" in event)
    print(f"Number of unique activities: {len(activities)}")
    print(f"Unique activities: {activities}")
    case_ids = [trace.attributes["concept:name"] for trace in log if "concept:name" in trace.attributes]
    print(f"Number of cases: {len(case_ids)}")
    print(f"First 5 case IDs: {case_ids[:5]}")
    print("\n\n")

except Exception as e:
    print(f"Error processing {filename}: {e}")


Processing MainProcess.xes


c:\Users\olive\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 301/301 [00:00<00:00, 691.70it/s]




Imported MainProcess.xes with 301 traces.
Number of events: 9471
Number of unique activities: 21
Unique activities: {'/ov/temper', '/sm/transport', '/ov/burn', '/hbw/store_empty_bucket', '/pm/punch_gill', '/hw/human_review', '/wt/pick_up_and_transport', '/mm/drill', '/pm/punch_ribbing', '/dm/lower', '/sm/sort', '/dm/cylindrical_drill', '/hbw/store', '/dm/drill', '/hbw/unload', '/mm/mill', '/mm/transport_from_to', '/hbw/get_empty_bucket', '/pm/punch_recesses', '/vgr/pick_up_and_transport', '/mm/deburr'}
Number of cases: 301
First 5 case IDs: ['WF_101_0', 'WF_102_0', 'WF_103_0', 'WF_104_0', 'WF_105_0']





Get all the attributes included in this file ("MainProcess.xes")

In [3]:
try:
    # List all attributes of the traces in a table
    print("\n\nAttributes of traces:\n")
    
    trace_attributes = set()
    for trace in log:
        trace_attributes.update(trace.attributes.keys())
        
    trace_attr_info = []
    for attr in trace_attributes:
        attr_type = "unknown"
        for trace in log:
            if attr in trace.attributes:
                attr_type = type(trace.attributes[attr]).__name__
                break
        trace_attr_info.append({"Attribute": attr, "Type": attr_type})

    trace_attr_df = pd.DataFrame(trace_attr_info)
    display(trace_attr_df)
    # Save the attribute DataFrame to an Excel file in a "tables" subfolder
    tables_dir = os.path.join(os.getcwd(), "tables")
    os.makedirs(tables_dir, exist_ok=True)
    trace_attr_df.to_excel(os.path.join(tables_dir, "main_trace_attribute_info.xlsx"), index=False)
    
    # List all attributes of the events in a table
    print("\n\nAttributes of events:\n")
    
    event_attributes = set()
    for trace in log:
        for event in trace:
            event_attributes.update(event.keys())

    event_attr_info = []
    for attr in event_attributes:
        attr_type = "unknown"
        for trace in log:
            for event in trace:
                if attr in event:
                    attr_type = type(event[attr]).__name__
                    break
            if attr_type != "unknown":
                break
        event_attr_info.append({"Attribute": attr, "Type": attr_type})

    event_attr_df = pd.DataFrame(event_attr_info)
    display(event_attr_df)
    # Save the attribute DataFrame to an Excel file in a "tables" subfolder
    tables_dir = os.path.join(os.getcwd(), "tables")
    os.makedirs(tables_dir, exist_ok=True)
    event_attr_df.to_excel(os.path.join(tables_dir, "main_event_attribute_info.xlsx"), index=False)
except Exception as e:
    print(f"Error processing the log: {e}")




Attributes of traces:



,Attribute,Type
0,concept:name,str




Attributes of events:



,Attribute,Type
0,current_task,str
1,case:concept:name,str
2,org:resource,str
3,response_status_code,float
4,human_workstation_green_button_pressed,float
5,identifier:id,str
6,lifecycle:state,str
7,unsatisfied_condition_description,str
8,parameters,dict
9,event_id,str


List all the resources of the log in a table

In [4]:
try:
    # Set pandas display options for full width
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 0)
    pd.set_option('display.max_columns', None)

    resources = set()
    for trace in log:
        for event in trace:
            if "org:resource" in event:
                resources.add(event["org:resource"])

    resource_info = []
    for resource in resources:
        # Find the first SubProcessID for this resource by searching events until found
        first_subprocess_id = "N/A"
        parameters_dict = None
        found = False
        for trace in log:
                for event in trace:
                    if event.get("org:resource") == resource:
                        if "SubProcessID" in event:
                            first_subprocess_id = event["SubProcessID"]
                            found = True
                            break
                if found:
                    break
                
        found = False
        for trace in log:
                for event in trace:
                    if event.get("org:resource") == resource:
                        if "parameters" in event:
                            parameters_dict = event["parameters"]
                            found = True
                            break
                if found:
                    break
        event_count = sum(1 for trace in log for event in trace if event.get("org:resource") == resource)
        activities_performed = set(event["concept:name"] for trace in log for event in trace if event.get("org:resource") == resource)
        
        resource_info.append({
            "Resource": resource,
            "Event Count": event_count,
            "Unique Activities": len(activities_performed),
            "Activities": ", ".join(sorted(activities_performed)),
        })

    resource_df = pd.DataFrame(resource_info)
    display(resource_df)
    # Save the attribute DataFrame to an Excel file in a "tables" subfolder
    tables_dir = os.path.join(os.getcwd(), "tables")
    os.makedirs(tables_dir, exist_ok=True)
    resource_df.to_excel(os.path.join(tables_dir, "resources_info.xlsx"), index=False)

except Exception as e:
    print(f"Error processing the log: {e}")

,Resource,Event Count,Unique Activities,Activities
0,pm_1,204,3,"/pm/punch_gill, /pm/punch_recesses, /pm/punch_ribbing"
1,sm_1,378,2,"/sm/sort, /sm/transport"
2,vgr_2,885,1,/vgr/pick_up_and_transport
3,ov_1,612,2,"/ov/burn, /ov/temper"
4,ov_2,330,1,/ov/burn
5,mm_1,576,4,"/mm/deburr, /mm/drill, /mm/mill, /mm/transport_from_to"
6,hw_1,522,1,/hw/human_review
7,mm_2,381,4,"/mm/deburr, /mm/drill, /mm/mill, /mm/transport_from_to"
8,hbw_1,873,4,"/hbw/get_empty_bucket, /hbw/store, /hbw/store_empty_bucket, /hbw/unload"
9,sm_2,309,2,"/sm/sort, /sm/transport"


Load a single subevent log file

In [5]:


# Process 0a0a7c16-85d9-48be-a7d5-32931240c337.xes file
filename = "0a0a7c16-85d9-48be-a7d5-32931240c337.xes"
file_path = os.path.join(xes_directory, filename)
print(f"Processing {filename}")

try:
    subevent_log = xes_importer.apply(file_path)
    print("\n\n")
    print(f"Imported {filename} with {len(subevent_log)} traces.")

    # Print important information about the subevent_log
    print(f"Number of events: {sum(len(trace) for trace in subevent_log)}")
    activities = set(event["concept:name"] for trace in subevent_log for event in trace if "concept:name" in event)
    print(f"Number of unique activities: {len(activities)}")
    print(f"Unique activities: {activities}")
    case_ids = [trace.attributes["concept:name"] for trace in subevent_log if "concept:name" in trace.attributes]
    print(f"Number of cases: {len(case_ids)}")
    print(f"First 5 case IDs: {case_ids[:5]}")
    print("\n\n")

except Exception as e:
    print(f"Error processing {filename}: {e}")

Processing 0a0a7c16-85d9-48be-a7d5-32931240c337.xes


parsing log, completed traces :: 100%|██████████| 1/1 [00:00<00:00, 133.96it/s]




Imported 0a0a7c16-85d9-48be-a7d5-32931240c337.xes with 1 traces.
Number of events: 5
Number of unique activities: 5
Unique activities: {'transporting the workpiece to the sorting machine', 'transporting the workpiece to the mill', 'ejecting the workpiece to the conveyor belt', 'transporting the workpiece to the ejection position', 'milling the workpiece'}
Number of cases: 0
First 5 case IDs: []





List all the attributes in this file

In [6]:
try:
    # List all attributes in the subevent_log in a table and display nicely in Jupyter Notebook
    all_attributes = set()
    for trace in subevent_log:
        all_attributes.update(trace.attributes.keys())
        for event in trace:
            all_attributes.update(event.keys())

    # Prepare attribute type information
    attr_info = []
    for attr in all_attributes:
        if attr in subevent_log[0].attributes:
            attr_type = type(subevent_log[0].attributes[attr]).__name__
        elif len(subevent_log[0]) > 0 and attr in subevent_log[0][0]:
            attr_type = type(subevent_log[0][0][attr]).__name__
        else:
            attr_type = "unknown"
        attr_info.append({"Attribute": attr, "Type": attr_type})

    # Display as a pandas DataFrame
    attr_df = pd.DataFrame(attr_info)
    display(attr_df)
    # Save the attribute DataFrame to an Excel file in a "tables" subfolder
    tables_dir = os.path.join(os.getcwd(), "tables")
    os.makedirs(tables_dir, exist_ok=True)
    attr_df.to_excel(os.path.join(tables_dir, "sub_attribute_info.xlsx"), index=False)

except Exception as e:
    print(f"Error processing the subevent_log: {e}")

,Attribute,Type
0,stream:datastream,dict
1,time:timestamp,datetime
2,org:resource,str
3,concept:name,str
4,operation_end_time,datetime
5,SubProcessID,str


Load the database file with the sensor data and query it using DuckDB

In [ ]:
import duckdb
import os
from pathlib import Path
import pandas as pd


# Make sure xes_directory is defined before this line
parquet_dir = os.path.join(xes_directory, "parquet")

# Connect to an in-memory DuckDB instance
con = duckdb.connect(database=':memory:')

# Collect all .parquet files in the directory
parquet_file = os.path.join(parquet_dir, "all_combined_new.parquet")

# Ensure there are files to process
if not os.path.exists(parquet_file):
    raise FileNotFoundError(f"No Parquet file found at {parquet_file}")

# Register all Parquet files as a single virtual table (view)
# IMPORTANT: DuckDB's parquet_scan expects a *list of strings* to be passed as a DuckDB list literal
# Use array syntax ['file1.parquet', 'file2.parquet', ...]
parquet_list_str = f"'{parquet_file}'"
query = f"CREATE VIEW sensor_data AS SELECT * FROM parquet_scan([{parquet_list_str}])"
con.execute(query)

# Print the number of rows in the sensor_data table
if False:
    row_count = con.execute("SELECT COUNT(*) FROM sensor_data").fetchone()[0]
    print(f"Number of rows in sensor_data: {row_count}")

# Print column names
if False:
    info_df = con.execute("PRAGMA table_info('sensor_data')").df()
    print("\nColumns and types in sensor_data:")
    print(info_df['name'])

# Show a sample of the data
if False:
    print("\nSample rows from sensor_data:")
    sample_df = con.execute("SELECT * FROM sensor_data LIMIT 5").df()
    display(sample_df)


# ------------------------
# Enter query here
# ------------------------

df_sensor_grouped = con.execute("""
    SELECT 
        "org:resource", "stream:observation", count(*) AS event_count
    FROM sensor_data
    WHERE "org:resource" = 'ov_1'
    GROUP BY "org:resource", "stream:observation"
    
""").df()

# ------------------------


# Display with full width

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
display(df_sensor_grouped)




,org:resource,stream:observation,event_count
0,ov_2,http://iot.uni-trier.de/FTOnto#MotorSpeed,18246
1,ov_2,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_2_Property_Current_State,18246
2,ov_2,http://iot.uni-trier.de/FTOnto#ValveOpen,18246
3,ov_2,http://iot.uni-trier.de/FTOnto#CompressorPowerLevel,18246
4,ov_2,http://iot.uni-trier.de/FTOnto#NFC_Tag_UID,1529
5,ov_2,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_2_Property_Current_Task_Elapsed_Seconds_Since_Start,18246
6,ov_2,http://iot.uni-trier.de/FTOnto#LightBarrierInterrupted,18246
7,ov_2,http://iot.uni-trier.de/FTOnto#PositionSwitchPressed,36492


Analyse some example process from MainProcess file

In [8]:
# Process MainProcess.xes file
filename = "MainProcess.xes"
file_path = os.path.join(xes_directory, filename)
print(f"Processing {filename}")

try:
    log = xes_importer.apply(file_path)
    print("\n\n")
    print(f"Imported {filename} with {len(log)} traces.")
    
    
    # Display only the concept:name attribute of the events in the first trace
    if len(log) > 0:
        first_trace = log[0]
        print(f"First trace case ID: {first_trace.attributes.get('concept:name', 'N/A')}")
        for idx, event in enumerate(first_trace):
            print(f"Event {idx+1}: concept:name = {event.get('concept:name', 'N/A')}, SubProcessID = {event.get('SubProcessID', 'N/A')}")
    else:
        print("Log is empty, no traces to display.")

    print("\n---\n")
    
    first_subprocess_id = "27958fc0-4484-41ff-9260-e76f8a83a7cd"
    file = first_subprocess_id + ".xes"
    file_path = os.path.join(xes_directory, file)
    
    print(f"Processing subprocess file: {file}")

    sub_log = xes_importer.apply(file_path)
    print(f"Imported {file} with {len(sub_log)} traces.")

    # Display only the concept:name attribute of the events in the first trace of the subprocess log
    if len(sub_log) > 0:
        first_sub_trace = sub_log[0]
        for idx, event in enumerate(first_sub_trace):
            print(f"Event {idx+1}: concept:name = {event.get('concept:name', 'N/A')}, org:resource = {event.get('org:resource', 'N/A')}")
    else:
        print("Subprocess log is empty, no traces to display.")

except Exception as e:
    print(f"Error processing {filename}: {e}")

Processing MainProcess.xes


parsing log, completed traces :: 100%|██████████| 301/301 [00:00<00:00, 677.33it/s]






Imported MainProcess.xes with 301 traces.
First trace case ID: WF_101_0
Event 1: concept:name = /hbw/unload, SubProcessID = N/A
Event 2: concept:name = /hbw/unload, SubProcessID = 27958fc0-4484-41ff-9260-e76f8a83a7cd
Event 3: concept:name = /hbw/unload, SubProcessID = N/A
Event 4: concept:name = /vgr/pick_up_and_transport, SubProcessID = N/A
Event 5: concept:name = /vgr/pick_up_and_transport, SubProcessID = 4d198444-6633-4218-b1f7-ca67ec666360
Event 6: concept:name = /vgr/pick_up_and_transport, SubProcessID = N/A
Event 7: concept:name = /hbw/store_empty_bucket, SubProcessID = N/A
Event 8: concept:name = /hbw/store_empty_bucket, SubProcessID = e73aa303-98af-407e-be4e-5ebdee33bc4d
Event 9: concept:name = /hbw/store_empty_bucket, SubProcessID = N/A
Event 10: concept:name = /vgr/pick_up_and_transport, SubProcessID = N/A
Event 11: concept:name = /vgr/pick_up_and_transport, SubProcessID = 0e7b5a4c-4c03-47b2-96fd-e401ed7fbca9
Event 12: concept:name = /vgr/pick_up_and_transport, SubProcessI

parsing log, completed traces :: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s]

Imported 27958fc0-4484-41ff-9260-e76f8a83a7cd.xes with 1 traces.
Event 1: concept:name = moving towards the slot 0, org:resource = hbw_2
Event 2: concept:name = picking up the bucket from the slot, org:resource = hbw_2
Event 3: concept:name = transporting the bucket to the conveyor belt, org:resource = hbw_2
Event 4: concept:name = dropping off the bucket at the conveyor belt, org:resource = hbw_2
Event 5: concept:name = transporting the bucket to the vacuum gripper robot crane jib, org:resource = hbw_2


Analyze Subprocess Data

In [9]:
df_sensor_grouped = con.execute("""
    SELECT 
        *
    FROM sensor_data
    WHERE "org:resource" = 'ov_1'
    AND "concept:name" = 'transporting the workpiece to the inside of the oven'
    AND "trace:SubProcessID" = '001618a4-1be3-406c-bd83-ebe5cff5e4f7'
    
""").df()

grouped = df_sensor_grouped.groupby(['stream:system', 'stream:observation'])

for name, group in grouped:
    proc_type = group['stream:procedure_type'].iloc[0]
    print(f"\nGroup: {name}")
    if proc_type == 'stream:continuous':
        # Convert stream:value to numeric if possible
        vals = pd.to_numeric(group['stream:value'], errors='coerce')
        min_val = vals.min()
        max_val = vals.max()
        print(f"  stream:procedure_type = stream:continuous, range: [{min_val}, {max_val}]")
    else:
        unique_vals = group['stream:value'].unique()
        print(f"  stream:procedure_type = {proc_type}, values: {unique_vals}")
        


Group: ('http://iot.uni-trier.de/FTOnto#OV_1', 'http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Temperature')
  stream:procedure_type = stream:continuous, range: [25.427, 25.427]

Group: ('http://iot.uni-trier.de/FTOnto#OV_1', 'http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_State')
  stream:procedure_type = stream:discrete, values: ['not ready']

Group: ('http://iot.uni-trier.de/FTOnto#OV_1', 'http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_Task_Elapsed_Seconds_Since_Start')
  stream:procedure_type = stream:continuous, range: [3.031237, 5.53083]

Group: ('http://iot.uni-trier.de/FTOnto#OV_1_Light_Barrier_5', 'http://iot.uni-trier.de/FTOnto#LightBarrierInterrupted')
  stream:procedure_type = stream:binary, values: ['1.0' '0.0']

Group: ('http://iot.uni-trier.de/FTOnto#OV_1_Motor_1', 'http://iot.uni-trier.de/FTOnto#MotorSpeed')
  stream:procedure_type = stream:continuous, range: [-512.0, -512.0]

Group: ('http://iot.uni-trier.de/FTOnto#OV_1_Positi

In [10]:
df_sensor_grouped = con.execute("""
    SELECT 
        *
    FROM sensor_data
    WHERE "org:resource" = 'pm_1'
    AND "concept:name" = 'transporting the workpiece to the punch'
    
""").df()

grouped = df_sensor_grouped.groupby(['stream:system', 'stream:observation'])

for name, group in grouped:
    proc_type = group['stream:procedure_type'].iloc[0]
    print(f"\nGroup: {name}")
    if proc_type == 'stream:continuous':
        # Convert stream:value to numeric if possible
        vals = pd.to_numeric(group['stream:value'], errors='coerce')
        min_val = vals.min()
        max_val = vals.max()
        print(f"  stream:procedure_type = stream:continuous, range: [{min_val}, {max_val}]")
    else:
        unique_vals = group['stream:value'].unique()
        print(f"  stream:procedure_type = {proc_type}, values: {unique_vals}")


Group: ('http://iot.uni-trier.de/FTOnto#PM_1', 'http://iot.uni-trier.de/StreamDataAnnotationOnto#PM_1_Property_Current_State')
  stream:procedure_type = stream:discrete, values: ['not ready']

Group: ('http://iot.uni-trier.de/FTOnto#PM_1', 'http://iot.uni-trier.de/StreamDataAnnotationOnto#PM_1_Property_Current_Task_Elapsed_Seconds_Since_Start')
  stream:procedure_type = stream:continuous, range: [0.093731, 124.781221]

Group: ('http://iot.uni-trier.de/FTOnto#PM_1_Capacitive_Sensor_6', 'http://iot.uni-trier.de/FTOnto#Electric_Field_Changed')
  stream:procedure_type = stream:binary, values: ['0.0' '1.0']

Group: ('http://iot.uni-trier.de/FTOnto#PM_1_Capacitive_Sensor_7', 'http://iot.uni-trier.de/FTOnto#Electric_Field_Changed')
  stream:procedure_type = stream:binary, values: ['0.0' '1.0']

Group: ('http://iot.uni-trier.de/FTOnto#PM_1_Capacitive_Sensor_8', 'http://iot.uni-trier.de/FTOnto#Electric_Field_Changed')
  stream:procedure_type = stream:binary, values: ['1.0']

Group: ('http://io

In [11]:
import networkx as nx
from collections import Counter
import math

df_sensor_grouped = con.execute("""
    SELECT 
        "trace:SubProcessID", "time:timestamp", "concept:name"
    FROM sensor_data
    WHERE "org:resource" = 'hbw_2'
      AND "concept:name" NOT ILIKE 'calibrating%'
    GROUP BY "trace:SubProcessID", "time:timestamp", "concept:name"
""").df()

# Group by SubProcessID and collect event names in order
subprocess_groups = df_sensor_grouped.groupby('trace:SubProcessID')

events = []

for sub_id, group in subprocess_groups:
    # Sort by timestamp and get event names
    event_names = group.sort_values('time:timestamp')['concept:name'].tolist()
    # Remove consecutive duplicates
    for i in range(1, len(event_names)):
        if event_names[i] != event_names[i-1] and (event_names[i-1], event_names[i]) not in events:
            events.append((event_names[i-1], event_names[i]))
            
            
print(f"Total unique directly-follows relations: {len(events)}")
num_concept_names = df_sensor_grouped['concept:name'].nunique()
print(f"Number of unique concept:name values: {num_concept_names}")
print("Directly-follows relations (event1 -> event2):")
for e1, e2 in events:
    print(f"  {e1} -> {e2}")
    



Total unique directly-follows relations: 38
Number of unique concept:name values: 26
Directly-follows relations (event1 -> event2):
  moving towards the slot 6 -> picking up the bucket from the slot
  picking up the bucket from the slot -> transporting the bucket to the conveyor belt
  transporting the bucket to the conveyor belt -> dropping off the bucket at the conveyor belt
  dropping off the bucket at the conveyor belt -> transporting the bucket to the vacuum gripper robot crane jib
  transporting the bucket to the high-bay warehouse crane jib -> transporting the bucket to the high bay warehouse crane jib
  transporting the bucket to the high bay warehouse crane jib -> picking up the bucket from the conveyor belt
  picking up the bucket from the conveyor belt -> transporting the bucket to the slot 5
  transporting the bucket to the slot 5 -> dropping off the bucket at the slot
  moving towards the slot 3 -> picking up the bucket from the slot
  moving towards the slot 5 -> picking 

Analyze Sensordata for synthetic Sensordata inclusion

In [ ]:
df_sensor_grouped = con.execute("""
    SELECT 
        "trace:SubProcessID", "time:timestamp", "concept:name", "stream:timestamp", "stream:value", "stream:observation", "org:resource", "operation_end_time", "stream:system"
    FROM sensor_data
    WHERE "org:resource" = 'ov_1'
    GROUP BY "trace:SubProcessID", "time:timestamp", "concept:name", "stream:timestamp", "stream:value", "stream:observation", "org:resource", "operation_end_time", "stream:system"
    ORDER BY "stream:timestamp"
""").df()

display(df_sensor_grouped)

# Convert timestamps to pandas datetime (be lenient with mixed/iso formats)
# use errors='coerce' to avoid exceptions for inconsistent formats and infer format
df_sensor_grouped["stream:timestamp"] = pd.to_datetime(
    df_sensor_grouped["stream:timestamp"], errors="coerce", infer_datetime_format=True, utc=True
)
df_sensor_grouped["time:timestamp"] = pd.to_datetime(
    df_sensor_grouped["time:timestamp"], errors="coerce", infer_datetime_format=True, utc=True
)

# Get individual sensor_kinds
sensor_kinds = df_sensor_grouped[["stream:observation", "stream:system"]].drop_duplicates()

# Add a table to add values for individual sensor kinds
all_sensor_event_stats = pd.DataFrame(columns=["stream:observation", "stream:system", "mean_std_dt", "mean_mean_dt"])


for (obs, sys) in sensor_kinds.itertuples(index=False):
    
    delta_df = df_sensor_grouped[
        (df_sensor_grouped["stream:observation"] == obs) &
        (df_sensor_grouped["stream:system"] == sys)
    ].copy()
    
    # Compute time deltas between sensor readings
    delta_df["delta"] = delta_df.groupby(
        ["trace:SubProcessID", "concept:name", "time:timestamp"]
    )["stream:timestamp"].diff().dt.total_seconds()

    # Aggregate deltas per event (verwende Standardabweichung statt Varianz)
    event_stats = delta_df.groupby(
        ["trace:SubProcessID", "concept:name", "time:timestamp"]
    ).agg(
        mean_dt=("delta", "mean"),
        std_dt=("delta", "std"),
        count=("delta", "count")
    ).reset_index()
    
    # Calculate the overall mean of std_dt and mean_dt
    mean_std_dt = event_stats["std_dt"].mean()
    mean_mean_dt = event_stats["mean_dt"].mean()
    
    # Add the values to the all_sensor_event_stats table
    # use .loc to append a new row safely (avoids deprecated DataFrame.append)
    all_sensor_event_stats.loc[len(all_sensor_event_stats)] = {
        "stream:observation": obs,
        "stream:system": sys,
        "mean_std_dt": mean_std_dt,
        "mean_mean_dt": mean_mean_dt
    }
    
display(all_sensor_event_stats)

# Save the attribute DataFrame to an Excel file in a "tables" subfolder
tables_dir = os.path.join(os.getcwd(), "tables")
os.makedirs(tables_dir, exist_ok=True)
all_sensor_event_stats.to_excel(os.path.join(tables_dir, "sensor_event_time_deltas.xlsx"), index=False)




,trace:SubProcessID,time:timestamp,concept:name,stream:timestamp,stream:value,stream:observation,org:resource,operation_end_time,stream:system
0,633d065f-96c0-4c4b-8112-302990575763,2021-06-23T15:34:59.623000,opening the oven door,2021-06-23T15:34:59.623000,0.0,http://iot.uni-trier.de/FTOnto#MotorSpeed,ov_1,2021-06-23T15:35:02.467000,http://iot.uni-trier.de/FTOnto#OV_1_Motor_1
1,633d065f-96c0-4c4b-8112-302990575763,2021-06-23T15:34:59.623000,opening the oven door,2021-06-23T15:34:59.623000,0.0,http://iot.uni-trier.de/FTOnto#ValveOpen,ov_1,2021-06-23T15:35:02.467000,http://iot.uni-trier.de/FTOnto#OV_1_Valve_7
2,633d065f-96c0-4c4b-8112-302990575763,2021-06-23T15:34:59.623000,opening the oven door,2021-06-23T15:34:59.623000,1.0,http://iot.uni-trier.de/FTOnto#PositionSwitchPressed,ov_1,2021-06-23T15:35:02.467000,http://iot.uni-trier.de/FTOnto#OV_1_Position_Switch_2
3,633d065f-96c0-4c4b-8112-302990575763,2021-06-23T15:34:59.623000,opening the oven door,2021-06-23T15:34:59.623000,0.109088,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_Task_Elapsed_Seconds_Since_Start,ov_1,2021-06-23T15:35:02.467000,http://iot.uni-trier.de/FTOnto#OV_1
4,633d065f-96c0-4c4b-8112-302990575763,2021-06-23T15:34:59.623000,opening the oven door,2021-06-23T15:34:59.623000,1.0,http://iot.uni-trier.de/FTOnto#LightBarrierInterrupted,ov_1,2021-06-23T15:35:02.467000,http://iot.uni-trier.de/FTOnto#OV_1_Light_Barrier_5
...,...,...,...,...,...,...,...,...,...
401652,86ce0424-998a-48d7-9041-3581d11e1352,2022-02-27T16:57:23.619000,transporting the workpiece to the outside of the oven,2022-02-27T16:57:26.351000,512.0,http://iot.uni-trier.de/FTOnto#MotorSpeed,ov_1,2022-02-27T16:57:26.351000,http://iot.uni-trier.de/FTOnto#OV_1_Motor_1
401653,86ce0424-998a-48d7-9041-3581d11e1352,2022-02-27T16:57:23.619000,transporting the workpiece to the outside of the oven,2022-02-27T16:57:26.351000,21.685378,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_Task_Elapsed_Seconds_Since_Start,ov_1,2022-02-27T16:57:26.351000,http://iot.uni-trier.de/FTOnto#OV_1
401654,86ce0424-998a-48d7-9041-3581d11e1352,2022-02-27T16:57:23.619000,transporting the workpiece to the outside of the oven,2022-02-27T16:57:26.351000,512.0,http://iot.uni-trier.de/FTOnto#CompressorPowerLevel,ov_1,2022-02-27T16:57:26.351000,http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Compressor_8
401655,86ce0424-998a-48d7-9041-3581d11e1352,2022-02-27T16:57:23.619000,transporting the workpiece to the outside of the oven,2022-02-27T16:57:26.351000,1.0,http://iot.uni-trier.de/FTOnto#ValveOpen,ov_1,2022-02-27T16:57:26.351000,http://iot.uni-trier.de/FTOnto#OV_1_Valve_7


C:\Users\olive\AppData\Local\Temp\ipykernel_22648\206678629.py:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_sensor_grouped["stream:timestamp"] = pd.to_datetime(
C:\Users\olive\AppData\Local\Temp\ipykernel_22648\206678629.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_sensor_grouped["time:timestamp"] = pd.to_datetime(


,stream:observation,stream:system,mean_std_dt,mean_mean_dt
0,http://iot.uni-trier.de/FTOnto#MotorSpeed,http://iot.uni-trier.de/FTOnto#OV_1_Motor_1,0.004546,0.205181
1,http://iot.uni-trier.de/FTOnto#ValveOpen,http://iot.uni-trier.de/FTOnto#OV_1_Valve_7,0.004546,0.205181
2,http://iot.uni-trier.de/FTOnto#PositionSwitchPressed,http://iot.uni-trier.de/FTOnto#OV_1_Position_Switch_2,0.004546,0.205181
3,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_Task_Elapsed_Seconds_Since_Start,http://iot.uni-trier.de/FTOnto#OV_1,0.004546,0.205181
4,http://iot.uni-trier.de/FTOnto#LightBarrierInterrupted,http://iot.uni-trier.de/FTOnto#OV_1_Light_Barrier_5,0.004546,0.205181
5,http://iot.uni-trier.de/FTOnto#CompressorPowerLevel,http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Compressor_8,0.004546,0.205181
6,http://iot.uni-trier.de/FTOnto#PositionSwitchPressed,http://iot.uni-trier.de/FTOnto#OV_1_Position_Switch_1,0.004546,0.205181
7,http://iot.uni-trier.de/StreamDataAnnotationOnto#OV_1_Property_Current_State,http://iot.uni-trier.de/FTOnto#OV_1,0.004546,0.205181
8,http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Temperature,http://iot.uni-trier.de/FTOnto#OV_1,0.015280,0.101384
9,http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Pneumatic_System_Pressure,http://iot.uni-trier.de/FTOnto#OV_1_WT_1_Compressor_8,0.012732,0.101391
